In [13]:
#imports
import requests #send HTTP requests to web pages and retrieve their content
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import pandas as pd
from nltk.corpus import stopwords
import string

In [14]:
import ssl
#ssl._create_default_https_context = ssl._create_unverified_context
#uncomment if you cannot properly download punkt

nltk.download('punkt')
#ssl._create_default_https_context = ssl._create_default_https_context
#uncomment if you cannot properly download punkt

#remove filler words from an example column
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/william/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/william/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

# Step 1: Define the search query and retrieve search results
search_query = input("What would you like to search up?")
search_engine_url = "https://www.google.com/search?q="

# Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Number of search result pages to retrieve (each page typically has 10 results)
num_pages = 2

search_results = []
for page in range(num_pages):
    # Calculate the start index for the current page
    start_index = page * 10

    # Make a GET request to retrieve the search result page
    response = requests.get(search_engine_url + search_query + f"&start={start_index}", headers=headers)

    # Step 2: Parse and extract information from the search result page
    soup = BeautifulSoup(response.content, "html.parser")

    for result in soup.select("div.g"):
        title_element = result.select_one("h3")
        if title_element:
            title = title_element.get_text()
            url_element = result.find("a")
            if url_element:
                url = url_element["href"]
                parsed_url = urlparse(url)
                if parsed_url.scheme == "":
                    # Relative URL, convert it to absolute URL
                    base_url = response.url
                    url = urljoin(base_url, url)

            # Make a GET request to the URL of the search result
            result_response = requests.get(url, headers=headers)

            # Extract visible text content from the HTML response
            result_soup = BeautifulSoup(result_response.content, "html.parser")
            text = result_soup.get_text()

            # Remove excess whitespace from the extracted text
            text = ' '.join(text.split())

            search_results.append({"Title": title, "URL": url, "Text": text})

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [16]:
df = pd.DataFrame(search_results)
print(len(df))
#unfiltered dataframe
df.head()

20


,Title,URL,Text
0,What is Artificial Intelligence (AI),https://www.ibm.com/topics/artificial-intellig...,What is Artificial Intelligence (AI) ? | IBM W...
1,Artificial Intelligence (AI): What Is AI and H...,https://builtin.com/artificial-intelligence,Artificial Intelligence (AI): What Is AI and H...
2,What is artificial intelligence (AI)? - AI def...,https://www.techtarget.com/searchenterpriseai/...,What is artificial intelligence (AI)? - AI def...
3,What is artificial intelligence?,https://www.brookings.edu/research/what-is-art...,What is artificial intelligence? Skip to main ...
4,What Is AI? Learn About Artificial Intelligence,https://www.oracle.com/artificial-intelligence...,What is Artificial Intelligence (AI)? | Oracle...


In [17]:
stop_words = stopwords.words("english") #english filler words

def remove_stopwords(text):
    '''
    Input a text block and removes all filler words. Returns the filtered text block in the form of a str
    '''
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

def remove_specialchars(text):
    '''
    Input a text block and removes all special characters. Returns the filtered text block in the form of a str
    '''
    cleaned_text = text.translate (str.maketrans('','', string.punctuation))
    return cleaned_text

In [18]:
example = df["Text"][0]
filtered_example = remove_stopwords(example)

print("length of unfiltered text: ", len(example))
print("length of filtered text: ", len(filtered_example))

length of unfiltered text:  14820
length of filtered text:  11851


In [19]:
#Properly using the apply() function onto a pandas column
#removes both filler words and special characters
df["Text"] = df["Text"].apply(remove_stopwords)
df["Text"] = df["Text"].apply(remove_specialchars)

In [20]:
print(len(df["Text"][0]))

11461


In [23]:
#filtered dataframe
df["Text"][0]

'Artificial Intelligence AI   IBM artificial intelligence AI Artificial intelligence leverages computers machines mimic problemsolving decisionmaking capabilities human mind Explore watsonx Sign AI updates artificial intelligence number definitions artificial intelligence AI surfaced last decades John McCarthy offers following definition 2004 paper PDF 127 KB link resides outside IBM  science engineering making intelligent machines especially intelligent computer programs related similar task using computers understand human intelligence AI confine methods biologically observable However decades definition birth artificial intelligence conversation denoted Alan Turings seminal work Computing Machinery Intelligence PDF 92 KB link resides outside IBM published 1950 paper Turing often referred father computer science asks following question Can machines think there offers test famously known Turing Test human interrogator would try distinguish computer human text response test undergone m